In [3]:
# default_exp migration

In [3]:
!which python
!python --version

/home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/python
Python 3.6.11


In [1]:
from villaProductSdk.products import ProductSdk
from nicHelper.dictUtil import printDict

ImportError: No module named villaProductSdk.products

In [77]:
psdk = ProductSdk(branch='dev-manual')

In [78]:
# Get all products

In [79]:
%%time
psdk.printFirst(psdk.allQuery())

CPU times: user 1.01 s, sys: 0 ns, total: 1.01 s
Wall time: 3.17 s


('0217153',
 {'0217153': {'cprcode': '0217153',
   'iprcode': '0217153',
   'oprcode': '0217153',
   'ordertype': 'Y',
   'pr_abb': 'COCOA LOCO MILK CHOC',
   'pr_active': 'Y',
   'pr_cgcode': '98',
   'pr_code': '0217153',
   'pr_dpcode': '28',
   'pr_engname': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.',
   'pr_ggcode': '003',
   'pr_market': 'COCOA LOCO MILK CHOCOLATE OWL',
   'pr_name': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.',
   'pr_puqty': '24',
   'pr_sa_method': '1',
   'pr_sucode1': 'F1222',
   'pr_suref3': 'S',
   'prtype': 'I',
   'psqty': '1',
   'pstype': '1'}})

## define dataclass for original dynamodb

In [89]:
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, JSONAttribute

class ProductMaster(Model):
  class Meta:
    table_name= 'villa-product-master'
    region = 'ap-southeast-1'
  iprcode = UnicodeAttribute(hash_key=True)
  cprcode = UnicodeAttribute(range_key=True)
  details = JSONAttribute()

### Get all items

In [117]:
items = [item.details for item in ProductMaster.scan(limit=100000)]
print(len(items))
print(items[:2])

68978
[{'cprcode': '0140942', 'iprcode': '0140942', 'oprcode': '0140942', 'ordertype': 'Y', 'pr_abb': 'FRAGATA OLIVE POMACE', 'pr_active': 'Y', 'pr_cgcode': '01', 'pr_code': '0140942', 'pr_dpcode': '08', 'pr_engname': 'FRAGATA OLIVE POMACE OIL 2 LITE', 'pr_ggcode': '008', 'pr_market': 'FRAGATA น้ำมันมะกอกธรรมชาติผสม', 'pr_name': 'FRAGATA น้ำมันมะกอกธรรมชาติ-กรรมวิธี 2 ลิตร', 'pr_puqty': '8.00', 'pr_sa_method': '1', 'pr_sucode1': '1036', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1', 'pr_country_th': '', 'pr_country_en': '', 'pr_keyword_th': '', 'pr_keyword_en': 'cooking oil, oil cooking,Fragata Olive Pomace Oil, Fragata, Olive purity pomace oil, Omace oil olive pomace oil, cooking olive pomace oil, olive pomace oil cooking\r\n', 'pr_filter_th': 'Oils  Vinegar,Grocery', 'pr_filter_en': 'Oils  Vinegar,Grocery', 'online_category_l1_th': 'สินค้าบริโภค', 'online_category_l1_en': 'Grocery', 'online_category_l2_th': 'น้ำมันและน้ำส้มสายชู', 'online_category_l2_en': 'Oils & Vineg

## Get rid of error item

In [118]:
def replaceEmptystringWithNone(inputListOfDict:list):
  newList = []
  for item in inputListOfDict:
    newItem = {k:v if v else 'none' for k,v in item.items()}
    newList.append(newItem)
  return newList
    

In [119]:
%%time
filledItems = replaceEmptystringWithNone(items)
len(filledItems)

68978

## update items with s3

In [121]:
%%time
psdk.updateWithS3(filledItems, invocationType='Event')

'successfully sent event, please watch your slack'